# 광주 인공지능 사관학교
- - -
- 작성자 : 2반 한지호
- 작성일 : 20.07.22 수
- 4교시 자연어 처리 시간에 진행한 나이브 베이즈 분류기 수업 자료
- - -
- 수정일 : 20.07.23 목
- 금일 진도에 따른 start_validate, open_csv_validate 함수 추가 작성

In [1]:
import re
import csv
from math import log, exp
from collections import Counter

In [2]:
# 최종 목표는 P(긍정|사용자가 입력한 값), P(부정|사용자가 입력한 값)
# P(긍정|사용자가 입력한 값) = P(사용자가 입력한 값|긍정) * P(긍정) / P(사용자가 입력한 값) -> 베이즈 정리
# 사용자가 입력한 값 = test_data
# 긍정 / 부정 (데이터셋) = train_data

def start():
    train_datas = open_csv() 
    test_data = input() 
    prob = naive_bayes(train_datas, test_data, 0.5, 0.5) 

    print(f'{test_data}가 부정적일 확률 : {prob[0]}, 긍정적일 확률 : {prob[1]}') 

In [3]:
def open_csv():
    f = open('IMDB Dataset.csv', 'r', encoding='utf-8') 
    csvreader = csv.reader(f) 

    pos_doc = []
    neg_doc = []

    next(csvreader) # 첫줄은 헤더라서 스킵
    
    for line in csvreader:
        if line[1] == 'positive':
            pos_doc.append(line[0])
        else:
            neg_doc.append(line[0])

    train_datas = [[], []]
    train_datas[0] = neg_doc
    train_datas[1] = pos_doc

    # 리스트 형태는 토큰화하기가 어렵기 때문에, 전부 조인을 통해서 하나의 문자열로 만들어준다
    return [' '.join(train_datas[0]), ' '.join(train_datas[1])]

In [4]:
# P(test_data|긍정)
# 동전 2개 던져서 앞면 앞면 -> 1/2 * 1/2
# 긍정인 데이터셋이 있다. '최고 최고' -> 최고의 빈도수 / 전체 단어의 빈도수 * 최고의 빈도수 / 전체 단어의 빈도수
# 어떤 문장 (test_data) -> 문장이 나올 확률 = 각 단어가 등장할 확률 다 곱한 것
def calculate_doc_prob(train_data, test_data, nowords_weight):
    # 스탑워드 제거
    sw_train_data = re.compile('[^\w]').sub(' ', train_data.lower())
    # 토큰화
    sw_train_token = sw_train_data.split()
    # Bow화 (단어 : 빈도수 형태)
    train_vector = dict(Counter(sw_train_token))

    # 스탑워드 제거
    sw_test_data = re.compile('[^\w]').sub(' ', test_data.lower())
    # 토큰화
    sw_test_token = sw_test_data.split()
    # Bow화 (단어 : 빈도수 형태)
    test_vector = dict(Counter(sw_test_token))

    log_prob = 0

    total_wc = len(sw_train_token)

    # log(P(test_data|긍정(train_data)))
    # 문장이 나올 확률 = 각 단어가 등장할 확률 다 곱한 것
    # 단어 10개로 이루어진 문장, 각 단어가 나올 확률이 (10/500000) -> 문장이 나올 확률은 0.00000000000000000000000000000000000000001024
    # 0.2e-50 -> 0으로 인식 -> 로그함수를 취하면 -50
    for word in test_vector:
        if word in train_vector:
            log_prob += log(train_vector[word]/total_wc)
        else:
            # train 데이터셋에 없는 단어가 나오면... 해당 단어가 나올 확률 추정못함
            # 없는 단어는 요정도~ 나왔다고 가정하자! -> nowords_weight
            log_prob += log(nowords_weight/total_wc)

    return log_prob

In [5]:
def naive_bayes(train_datas, test_data, pos_prob, neg_prob):
    # P(긍정|test_data) = P(test_data|긍정) * P(긍정) / P(test_data)
    test_pos_prob = calculate_doc_prob(train_datas[1], test_data, 0.1) + log(pos_prob) 
    # P(부정|test_data) = P(test_data|부정) * P(부정) / P(test_data)
    test_neg_prob = calculate_doc_prob(train_datas[0], test_data, 0.1) + log(neg_prob) 

    # 10 : 5 -> 2 : 1
    # 긍정, 부정의 상대적인 크기
    # test_pos_prob (로그값) -> 로그값에서 지수값으로 변환해도 0
    # test_neg_prob (로그값) -> 로그값에서 지수값으로 변환해도 0
    # 로그함수 -> logex, 지수함수 -> e**x
    # -200, -220 -> 0, 0 (지수화하면 너무 작은 값이라 컴퓨터가 0으로 인식하기 때문에 손실이 생김) 
    # -200, -220 -> 0, -20 -> (지수화) e**0 = 1, e**-20 = 0.000000001
    # 로그값 != 확률, -13, -12 -> e배이상의 확률차이가난다는것 -13/(-13+-12), -12/(-13+-12)
    # -13, 12 -> 1, 0.35 -> 1/1.35, 0.35/1.35
    maxprob = max(test_neg_prob, test_pos_prob)
    test_pos_prob -= maxprob
    test_neg_prob -= maxprob
    test_pos_prob = exp(test_pos_prob)
    test_neg_prob = exp(test_neg_prob)
    # 두 확률 값의 상대적인 비율
    normalized_prob = [test_neg_prob/(test_pos_prob+test_neg_prob), test_pos_prob/(test_pos_prob+test_neg_prob)]

    return normalized_prob

In [6]:
start() 

so bad
so bad가 부정적일 확률 : 0.8345935948229825, 긍정적일 확률 : 0.16540640517701757


In [7]:
def start_validate():
    train_datas, test_datas = open_csv_validate()

    # 트레인 데이터 (긍정, 부정) 을 바탕으로 긍정적인 문장을 검사 -> 긍정적인 확률 높으면
    prob = naive_bayes(train_datas, test_datas[1], 0.5, 0.5)

    print(f'긍정적인 문장이 부정적일 확률 : {prob[0]}, 긍정적일 확률 : {prob[1]}')

    # 트레인 데이터 (긍정, 부정) 을 바탕으로 부정적인 문장을 검사 -> 부정적인 확률 높으면
    prob = naive_bayes(train_datas, test_datas[0], 0.5, 0.5)

    print(f'부정적인 문장이 부정적일 확률 : {prob[0]}, 긍정적일 확률 : {prob[1]}')

In [8]:
def open_csv_validate():
    f = open('IMDB Dataset.csv', 'r', encoding='utf-8')
    csvreader = csv.reader(f)

    pos_doc = []
    neg_doc = []

    next(csvreader)
    for line in csvreader:
        if line[1] == 'positive':
            pos_doc.append(line[0])
        else:
            neg_doc.append(line[0])

    train_datas = [[], []]
    # 25000개에서 20000 : 5000
    train_datas[0] = neg_doc[:20000]
    train_datas[1] = pos_doc[:20000]
    test_datas = [neg_doc[20000:], pos_doc[20000:]]

    # 리스트 형태는 토큰화하기가 어렵기 때문에, 전부 조인을 해서 하나의 문자열로 만들어준다
    return [' '.join(train_datas[0]), ' '.join(train_datas[1])], [' '.join(test_datas[0]), ' '.join(test_datas[1])]

In [9]:
start_validate() 

긍정적인 문장이 부정적일 확률 : 0.0, 긍정적일 확률 : 1.0
부정적인 문장이 부정적일 확률 : 1.0, 긍정적일 확률 : 0.0
